# FAISS DB
- Facebook AI Similarity Search

In [1]:
!pip install faiss-cpu
# !pip install faiss-gpu # gpu탑재시 

   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ------ --------------------------------- 2.4/13.7 MB 11.2 MB/s eta 0:00:02
   ------------- -------------------------- 4.7/13.7 MB 11.9 MB/s eta 0:00:01
   -------------------- ------------------- 7.1/13.7 MB 11.8 MB/s eta 0:00:01
   --------------------------- ------------ 9.4/13.7 MB 11.7 MB/s eta 0:00:01
   -------------------------------- ------- 11.3/13.7 MB 10.8 MB/s eta 0:00:01
   ---------------------------------------- 13.7/13.7 MB 10.9 MB/s eta 0:00:00


### 영화 추천 시스템

In [6]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')


In [5]:
movies = [
    {'id': 0, 'title': '인셉션', 'description': '꿈속에서 다른 사람의 생각을 조작하는 스토리'},
    {'id': 1, 'title': '인터스텔라', 'description': '우주를 여행하며 인류의 미래를 찾는 이야기'},
    {'id': 2, 'title': '기생충', 'description': '가난한 가족이 부잣집에 스며들며 벌어지는 사건'},
    {'id': 3, 'title': '올드보이', 'description': '15년간 감금된 남자가 복수를 위해 나서는 이야기'},
    {'id': 4, 'title': '어벤져스: 엔드게임', 'description': '어벤져스가 타노스를 막고 우주를 구하려는 이야기'},
    {'id': 5, 'title': '다크 나이트', 'description': '배트맨과 조커의 숨막히는 대결'},
    {'id': 6, 'title': '쇼생크 탈출', 'description': '억울한 누명을 쓰고 감옥에 갇힌 남자의 탈출 이야기'},
    {'id': 7, 'title': '포레스트 검프', 'description': '순수한 마음을 가진 남자가 겪는 감동적인 인생 이야기'},
    {'id': 8, 'title': '라라랜드', 'description': '꿈을 좇는 연인들의 아름다운 사랑 이야기'},
    {'id': 9, 'title': '타이타닉', 'description': '호화 여객선에서 벌어지는 운명적인 사랑 이야기'},
    {'id': 10, 'title': '매트릭스', 'description': '가상현실과 인간의 자유 의지를 다룬 SF 영화'},
    {'id': 11, 'title': '위대한 쇼맨', 'description': '서커스단을 창립한 한 남자의 감동적인 이야기'},
    {'id': 12, 'title': '해리 포터와 마법사의 돌', 'description': '마법 학교에서 펼쳐지는 소년 마법사의 성장 이야기'},
    {'id': 13, 'title': '어바웃 타임', 'description': '시간여행을 통해 사랑과 삶을 배우는 이야기'},
    {'id': 14, 'title': '토이 스토리', 'description': '장난감들이 벌이는 모험과 우정 이야기'},
    {'id': 15, 'title': '아이언맨', 'description': '억만장자 기술자가 히어로가 되어 세상을 지키는 이야기'},
    {'id': 16, 'title': '설국열차', 'description': '빙하기가 된 지구에서 생존자들이 기차 안에서 벌이는 이야기'},
    {'id': 17, 'title': '반지의 제왕: 반지 원정대', 'description': '절대 반지를 파괴하기 위한 모험과 전투 이야기'},
    {'id': 18, 'title': '킬 빌', 'description': '죽음에서 살아남은 여자의 복수극'},
    {'id': 19, 'title': '스타워즈: 새로운 희망', 'description': '은하를 구하기 위한 반란군과 제국의 대결'}
]

In [9]:

descriptions = [movie['description'] for movie in movies]
desc_embed = np.array([model.encode(description) for description in descriptions], dtype='float32') # chroma는 리스트의 형태를 요구하지만 face는 ndarry형식을 요구

In [10]:
dim = desc_embed.shape[1]
idx = faiss.IndexFlatL2(dim)    # chroma의 collection(=table)과 유사 : data를 가지고 있지만 구조에 더 가까운 의미를 가짐 
idx.add(desc_embed)


In [13]:
query_text = '탈출하는 이야기'
query_embed = np.array([model.encode('query_text')],dtype='float32')


In [16]:
top_n=5
distances, indeices = idx.search(query_embed, top_n)

In [17]:
print(distances)
print(indeices)

[[1.6643693 1.772795  1.7739822 1.7780049 1.8014685]]
[[ 3  0  2  4 14]]


In [57]:
for i in range(top_n):
    movie_id = indeices[0][i]
    # print(movies[movie_id]['title'])
    print(f'{i+1}번째 {movies[movie_id]['title']} | 유사도 거리: {distances[0][i]:.2f}')

SyntaxError: f-string: unmatched '[' (3314053379.py, line 4)

### 사용자 맞춤 뉴스 추천

In [65]:
import requests
from bs4 import BeautifulSoup

url = 'https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query=%EC%A0%95%EC%B9%98+%EB%89%B4%EC%8A%A4'

response = requests.get(url)

html = response.text
soup =  BeautifulSoup(html, 'html.parser')

news = soup.select_one('#main_pack > section > div.api_subject_bx > div.group_news > ul')
news_titles = news.select('li > div > div > div > a')

title_list = []
for news_title in news_titles:
    title =news_title.get_text()
    title_list.append(title)
    
for i, title in enumerate(title_list):
    if title == '':
       title_list.pop(i)
       
title_list



['동아일보 유튜브 라이브 ‘정치를 부탁해’ 10일 첫 방송',
 '주간 정치 핫뉴스…개헌론과 명태균특검법의 향방은? [한강로 AI정치데...',
 '다른 정치 성향 뉴스에…진보는 ‘싫어요’, 보수는 ‘반박 댓글’',
 '김민하 “한동훈, 계엄이 장난? 정치적으로 이용 말길” [김은지의 뉴스...',
 '[교육 돌아보기]유명 강사의 정치적 한마디, 그 무게감에 관하여',
 '중공중앙정치국, 시진핑 총서기 주재로 회의 열어 정부업무보고 논의',
 '[데스크 칼럼] 알고리즘 이겨내는 생활정치',
 "'정치인 꼴 보기 싫다'…예민한 사람들 왜 그런가 봤더니 [이슈+]",
 '중앙정치국 위원-서기처 서기 등 당 중앙 및 시진핑 총서기에게 업무 보...',
 '[손연우의 정치眼] 좋은 정치']

In [66]:

descriptions = [title for title in title_list]
desc_embed = np.array([model.encode(description) for description in descriptions], dtype='float32') # chroma는 리스트의 형태를 요구하지만 face는 ndarry형식을 요구

In [81]:
dim = desc_embed.shape[1]
idx = faiss.IndexFlatL2(dim)    # chroma의 collection(=table)과 유사 : data를 가지고 있지만 구조에 더 가까운 의미를 가짐 
idx.add(desc_embed)

In [82]:
query_text = '한동훈'
query_embed = np.array([model.encode('query_text')],dtype='float32')

In [83]:
top_n=3
distances, indeices = idx.search(query_embed, top_n)

In [84]:
print(distances)
print(indeices)

[[1.6792383 1.6918894 1.8351521]]
[[3 2 7]]


In [85]:
for i in range(top_n):
    title_id = indeices[0][i]
    # print(movies[movie_id]['title'])
    print(f'{i+1}번째 {title_list[title_id]} | 유사도 거리: {distances[0][i]:.2f}')

1번째 김민하 “한동훈, 계엄이 장난? 정치적으로 이용 말길” [김은지의 뉴스... | 유사도 거리: 1.68
2번째 다른 정치 성향 뉴스에…진보는 ‘싫어요’, 보수는 ‘반박 댓글’ | 유사도 거리: 1.69
3번째 '정치인 꼴 보기 싫다'…예민한 사람들 왜 그런가 봤더니 [이슈+] | 유사도 거리: 1.84
